In [40]:
from junifer.storage import HDF5FeatureStorage
from julearn.api import run_cross_validation
from julearn.pipeline import PipelineCreator
from julearn.viz import plot_scores
from julearn.stats.corrected_ttest import corrected_ttest
import pandas as pd
import seaborn as sns
from sklearn.svm import LinearSVC
import warnings


In [41]:
storage_parcels = HDF5FeatureStorage(uri='data/AOMICID1000_vbm_parcellations_sch100.hdf5')
storage_hists = HDF5FeatureStorage(uri='data/AOMICID1000_vbm_histogram.hdf5')

In [42]:
df_parcellations = storage_parcels.read_df('VBM_GM_Schaefer100x17_counts_aggregation')
df_histograms = storage_hists.read_df('VBM_GM_hist_hist')
df_demographics = pd.read_csv('data/participants.tsv',sep='\t')
df_demographics.rename(columns={"participant_id": "subject"}, inplace=True)

In [65]:
df_parcellations.columns = df_parcellations.columns.astype(str)
df_histograms.columns = df_histograms.columns.astype(str)

# List of columns for each dataframe
X_parcellations = list(df_parcellations.columns)
X_histograms = list(df_histograms.columns)

# Merge with df_demographics on 'subject'
df_full_parcellations = df_parcellations.merge(df_demographics, on="subject")
df_full_histograms = df_histograms.merge(df_demographics, on="subject")

# Map 'sex' column
df_full_parcellations['sex'] = df_full_parcellations['sex'].map({'F': 1, 'M': -1})
df_full_histograms['sex'] = df_full_histograms['sex'].map({'F': 1, 'M': -1})

# Drop rows with NaN values
df_full_parcellations = df_full_parcellations.dropna()
df_full_histograms = df_full_histograms.dropna()

sex_counts = df_full_parcellations['sex'].value_counts()
sex_counts


sex
 1.0    128
-1.0     96
Name: count, dtype: int64

In [47]:
creator = PipelineCreator(problem_type="classification")
creator.add("zscore")
creator.add(
    "svm",
    C=(0.001, 100, "log-uniform")
)

In [59]:
search_params = {
    "kind": "optuna",
    "cv":4
}

scoring = ["balanced_accuracy", "accuracy"]
scores_hists, model_hists, inspector_hists = run_cross_validation(
    X=X_histograms,
    y='sex',
    data=df_full_histograms,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring = scoring,
)
scores_schaefer, model_schaefer, inspector_schaefer = run_cross_validation(
    X=X_parcellations,
    y='sex',
    data=df_full_parcellations,
    search_params=search_params,
    model=creator,
    return_train_score=True,
    return_inspector=True,
    cv=4,
    scoring = scoring,
)

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/prepare.py:505: RuntimeWarning: The following columns are not defined in X_types: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']. They will be treated as continuous.
  warn_with_log(

/home/hsreekri/Hsreekri/lib/python3.11/site-packages/julearn/pipeline/pipeline_creator.py:1041: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the futur

In [60]:
scores_schaefer

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.323450,0.003514,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.5,0.5,0.589286,0.565476,168,56,0,0,bc7087515161a73a5a6aff57863f3803
1,0.318093,0.003509,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.5,0.5,0.517857,0.589286,168,56,0,1,bc7087515161a73a5a6aff57863f3803
2,0.322214,0.003567,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.5,0.5,0.642857,0.547619,168,56,0,2,bc7087515161a73a5a6aff57863f3803
3,0.313486,0.003461,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.5,0.5,0.535714,0.583333,168,56,0,3,bc7087515161a73a5a6aff57863f3803


In [61]:
scores_hists

,fit_time,score_time,estimator,test_balanced_accuracy,train_balanced_accuracy,test_accuracy,train_accuracy,n_train,n_test,repeat,fold,cv_mdsum
0,0.345349,0.003595,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.500000,0.500000,0.464286,0.607143,168,56,0,0,bc7087515161a73a5a6aff57863f3803
1,0.337329,0.003577,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.500000,0.500000,0.517857,0.589286,168,56,0,1,bc7087515161a73a5a6aff57863f3803
2,0.330072,0.003487,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.425000,0.947159,0.446429,0.946429,168,56,0,2,bc7087515161a73a5a6aff57863f3803
3,0.338125,0.005270,"OptunaSearchCV(cv=KFold(n_splits=4, random_sta...",0.483531,0.974189,0.500000,0.976190,168,56,0,3,bc7087515161a73a5a6aff57863f3803


In [62]:
scores_hists['model'] = 'AOMIC_Histograms'
scores_schaefer['model'] = 'AOMIC_Schaefer'
plot_scores(scores_schaefer,scores_hists)


BokehModel(combine_events=True, render_bundle={'docs_json': {'bacad294-0994-47fa-88ee-072b60b94cb5': {'version…

In [63]:
stats_df = corrected_ttest(scores_schaefer,scores_hists)
print(stats_df)

                    metric    t-stat     p-val         model_1  \
0   test_balanced_accuracy  0.840742  0.462219  AOMIC_Schaefer   
1  train_balanced_accuracy -1.132919  0.339605  AOMIC_Schaefer   
2            test_accuracy  1.318124  0.279071  AOMIC_Schaefer   
3           train_accuracy -1.255932  0.298050  AOMIC_Schaefer   

            model_2  p-val-corrected  
0  AOMIC_Histograms         0.462219  
1  AOMIC_Histograms         0.339605  
2  AOMIC_Histograms         0.279071  
3  AOMIC_Histograms         0.298050  
